# Add Dependences

In [6]:
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/run_event.jl")

@make_struct_func Cubic

force_func (generic function with 1 method)

# Add Data

In [7]:
parameters = Model(
    Cubic(
        μ₁       = 0.1,
        rₘᵢₙ     = 2,
        rₘₐₓ     = 3
    ),
    Contractile(
        fₚ       = 0.0000001
    ),
    Time(
        t_f      = 100000, 
        dt       = 0.1
    ),
    Neighbor(
        n_knn    = 50,
        nn       = 12
    ),
    Geometry(
        R_agg    = 9,
        position = [
            0.0    0.0    0.0
        ]
    ),
    Simulation(
        n_text   = 200,
        path     = "Test.xyz"
    )
)

Model(Cubic(0.1, 2.0, 3.0), Contractile(1.0e-7), Time(100000.0, 0.1), Neighbor(50, 12), Geometry(9.0, [0.0 0.0 0.0]), Simulation(200, "Test.xyz"))

In [8]:
println("Loading Data")
CUDA.@time X = Float32.(readdlm("../../data/init/Sphere/$(Int(parameters.Geometry.R_agg)).xyz")[3:end,2:end]) |> cu
println("\n Generating Struct")
CUDA.@time Agg = Aggregate(Neighbor(50, 12), PositionCell(X));

## Var before func
println("\n Loading Force Function")
CUDA.@time force(r) =  force_func(parameters.Force::Cubic, r)

## Adding Sum of Aggregates
println("\n Adding Aggregates from Position")
CUDA.@time SumAgg(Agg::Aggregate, parameters::Model)

# Simulating
println("\n Simulation")
CUDA.@time simulate(true, parameters, Cubic(1.0, 2.0, 3.0), Agg)

Loading Data
  0.774219 seconds (2.55 M CPU allocations: 148.087 MiB, 3.54% gc time) (1 GPU allocation: 6.305 KiB, 8.36% memmgmt time)

 Generating Struct
  0.915642 seconds (2.30 M CPU allocations: 134.801 MiB, 3.08% gc time) (12 GPU allocations: 4.813 MiB, 0.01% memmgmt time)

 Loading Force Function
  0.000024 seconds (24 CPU allocations: 1.634 KiB)

 Adding Aggregates from Position
 14.415989 seconds (50.37 M CPU allocations: 2.842 GiB, 6.71% gc time) (20 GPU allocations: 4.850 MiB, 0.00% memmgmt time)

 Simulation


Progress: 100%|█████████████████████████| Time: 0:03:35m:25


215.478274 seconds (759.66 M CPU allocations: 40.708 GiB, 5.26% gc time) (12.70 M GPU allocations: 1.363 TiB, 10.81% memmgmt time)
